In [1]:
!pip install mahotas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 10.7 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import mahotas

from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [3]:
'''
    IMAGE UTIL FUNCTIONS
'''

def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    
    if width is None:
        # new height/old height
        r = height / float(h)
        dim = (int(w * r), height) # dimension is in the form of (width, height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [4]:
'''
HOG FEATURE DESCRIPTOR
'''
from skimage import feature

class HOG:
    def __init__(self, orientations=9, pixelsPerCell=(8, 8), cellsPerBlock=(3, 3), transform=False):

        self.orientations = orientations  # number of bins
        self.pixelsPerCell = pixelsPerCell  # number of pixels per cell
        self.cellsPerBlock = cellsPerBlock  # number of cells per each block
        self.transform = transform

    def describe(self, image):
        hist = feature.hog(image,
                           orientations=self.orientations,
                           pixels_per_cell=self.pixelsPerCell,
                           cells_per_block=self.cellsPerBlock,
                           transform_sqrt=self.transform)
        return hist

In [5]:
'''
    UTILITIES TO PREPARE AND PROCESS DATA
'''
def load_digits(datasetPath):
    data = np.genfromtxt(datasetPath, delimiter=",", dtype="uint8")
    target = data[:, 0]
    data = data[:, 1:].reshape(data.shape[0], 28, 28)

    return (data, target)


def deskew(image, width):
    (h, w) = image.shape[:2]
    # moments contain the statistical information regarding the distribution of location of white pixels in image
    moments = cv2.moments(image)

    skew = moments["mu11"]/moments["m02"]
    M = np.float32([
        [1, skew, -0.5*w*skew],
        [0, 1, 0]
    ])

    image = cv2.warpAffine(image, M, (w, h), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)

    image = resize(image, width=width)

    return image


def center_extent(image, size):
    '''
        Arguments:
            image: deskewed image
            size: output size of image i.e. output width and height
    '''
    (eW, eH) = size
    # if the width is greater than the height of image
    if image.shape[1] > image.shape[0]:
        # Resize the image based on its width
        image = resize(image, width=eW)
    else:
        # If the height is greater than width then resize the image based on its height
        image = resize(image, height=eH)

    extent = np.zeros((eW, eH), dtype="uint8")

    offsetX = (eW - image.shape[1]) // 2
    offsetY = (eH - image.shape[0]) // 2
    extent[offsetY: offsetY + image.shape[0],
           offsetX: offsetX + image.shape[1]] = image
    # Compute the weighted mean of white pixels in the image using center of mass function
    # Returns the weighted x,y coordinates of center of image
    CM = mahotas.center_of_mass(extent)
    # Convert the x,y coordinates to integers rather than floats
    (cY, cX) = np.round(CM).astype("int32")
    (dX, dY) = ((size[0] // 2) - cX, (size[1] // 2) - cY)

    M = np.float32([[1, 0, dX], [0, 1, dY]])
    extent = cv2.warpAffine(extent, M, size)

    return extent

In [6]:
train_data_path = "/kaggle/input/digit-recognizer/train.csv"
test_data_path =  "/kaggle/input/digit-recognizer/test.csv"
output = "submission.csv"

In [7]:
# Load the data
X, y = load_digits(train_data_path)

In [8]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
# print('Number of training samples: ', len(X_train))
# print('Number of validation samples: ',len(X_val))

In [10]:
# Instantiate the feature descriptor
hog = HOG(orientations=18, pixelsPerCell=(10, 10), cellsPerBlock=(1, 1), transform=True)

In [11]:
# Apply feature descriptor to training images
X_train_desc = []
for image in X:
    image = deskew(image, 20)
    image = center_extent(image, (20, 20))

    hist = hog.describe(image)
    X_train_desc.append(hist)

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [13]:
model = XGBClassifier(random_state=42)

In [14]:
print('Training the model...')
model.fit(X_train_desc, y)
print('Training completed!')

Training the model...
Training completed!


In [15]:
print('Loading Test data...')
X_test = np.loadtxt(test_data_path, skiprows=1, dtype='int', delimiter=',')
X_test = X_test.astype("float32")
X_test = X_test.reshape(X_test.shape[0], 28, 28)
print('Test data loaded!')

Loading Test data...
Test data loaded!


In [16]:
len(X_test)

28000

In [17]:
# Apply feature descriptor to test data
print('Processing Test data...')
X_test_desc = []
for image in X_test:
    image = deskew(image, 20)
    image = center_extent(image, (20, 20))

    hist = hog.describe(image)
    X_test_desc.append(hist)

Processing Test data...


In [18]:
print('Generating predictions for Test data...')
y_pred = model.predict(X_test_desc)

Generating predictions for Test data...


In [19]:
# print(classification_report(y_val, y_pred))

In [20]:
with open(output, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))
    print('Submissions file created!')

Submissions file created!
